In [3]:
import utils
import kaolin as kal
import torch
import numpy as np
import ipyvolume as ipv
from PIL import Image

#mesh = kal.rep.TriangleMesh.from_obj("model.obj") # Modelo de una esfera
mesh = kal.rep.TriangleMesh.from_obj("cow.obj") # Modelo de vaca
normals = utils.compute_vertex_normals(mesh)
utils.visualize_normals(mesh, normals)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.7761263…

In [5]:
import pointareas

pa = pointareas.calculate_pointareas(mesh)
pointareas.visualize_pointareas(mesh, pa)

f = open("model_pointareas", "r")
tri_pa = [float(l) for l in f]
tri_pa_tens = torch.tensor(tri_pa)
pointareas.visualize_pointareas(mesh, tri_pa_tens)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.7761263…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.7761263…

In [15]:
def norm_sq(v, dim):
    return torch.sum(v**2, dim=dim)

def calculate_pointareas(mesh):
    verts = mesh.vertices
    faces = mesh.faces

    # Conseguir vectores de bordes
    e0 = verts[faces[:,2]] - verts[faces[:,1]]
    e1 = verts[faces[:,0]] - verts[faces[:,2]]
    e2 = verts[faces[:,1]] - verts[faces[:,0]]

    # Computar los pesos en cada esquina, basados en el area
    area = 0.5 * torch.norm(torch.cross(e0, e1), dim=1)
    l2 = torch.stack([norm_sq(e0, 1), norm_sq(e1, 1), norm_sq(e2, 1)], dim=1)

    # Pesos baricéntricos del circumcentro
    bcw = torch.stack([l2[:,0] * (l2[:,1] + l2[:,2] - l2[:,0]),
                       l2[:,1] * (l2[:,2] + l2[:,0] - l2[:,1]),
                       l2[:,2] * (l2[:,0] + l2[:,1] - l2[:,2])], dim=1)

    # Calculo las areas en las esquinas en base a los pesos baricéntricos
    pointareas = torch.zeros(verts.shape, dtype=verts.dtype)
    cornerareas = torch.zeros(faces.shape, dtype=verts.dtype)

    for i in range(0, faces.shape[0]):
            if bcw[i,0] <= 0.0:
                    cornerareas[i,1] = -0.25 * l2[i,2] * area[i] / torch.dot(e0[i], e2[i])
                    cornerareas[i,2] = -0.25 * l2[i,1] * area[i] / torch.dot(e0[i], e1[i])
                    cornerareas[i,0] = area[i] - cornerareas[i,1] - cornerareas[i,2]

            elif bcw[i,1] <= 0.0:
                    cornerareas[i,2] = -0.25 * l2[i,0] * area[i] / torch.dot(e1[i], e0[i])
                    cornerareas[i,0] = -0.25 * l2[i,2] * area[i] / torch.dot(e1[i], e2[i])
                    cornerareas[i,1] = area[i] - cornerareas[i,2] - cornerareas[i,0]

            elif bcw[i,2] <= 0.0:
                    cornerareas[i,0] = -0.25 * l2[i,1] * area[i] / torch.dot(e2[i], e1[i])
                    cornerareas[i,1] = -0.25 * l2[i,0] * area[i] / torch.dot(e2[i], e0[i])
                    cornerareas[i,2] = area[i] - cornerareas[i,0] - cornerareas[i,1]

            else:
                    scale = 0.5 * area[i] / (bcw[i,0] + bcw[i,1] + bcw[i,2])
                    cornerareas[i,0] = scale * (bcw[i,1] + bcw[i,2])
                    cornerareas[i,1] = scale * (bcw[i,2] + bcw[i,0])
                    cornerareas[i,2] = scale * (bcw[i,0] + bcw[i,1])

            pointareas[faces[i,0]] += cornerareas[i,0]
            pointareas[faces[i,1]] += cornerareas[i,1]
            pointareas[faces[i,2]] += cornerareas[i,2]

    return pointareas

In [16]:
pa = calculate_pointareas(mesh)
print(pa)

tensor([[0.0214, 0.0214, 0.0214],
        [0.0353, 0.0353, 0.0353],
        [0.0247, 0.0247, 0.0247],
        ...,
        [0.0046, 0.0046, 0.0046],
        [0.0038, 0.0038, 0.0038],
        [0.0057, 0.0057, 0.0057]])
